# 통계 데이터 분석 HW2

In [14]:
import numpy as np
from scipy import stats
from scipy.stats import shapiro
import scipy.stats

import pandas as pd
import statsmodels.api as sm
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest

%matplotlib inline

matplotlib.rc('font', family = 'Malgun Gothic')
plt.rc('font', family = 'Malgun Gothic')

### 와이블 분포 : 어떤 제품의 수명시간 x가 형상모수 5.2, 척도모수 2,200인 와이블 분포를 따른다고 한다. 이 제품이 적어도 2,50 시간 이상 작동할 확률은?

In [15]:
x = 2500
alpha = 5.2
beta = 2200

prob = stats.weibull_min.cdf(x, alpha, scale = beta)
print("P(X<=x):{:.3f}".format(1 - prob))

P(X<=x):0.143


### 이항 분포 : 도장 공정에서 광택도 불량이 30%나 된다고 한다. 5대의 차량을 임의로 선택했을 때 불량대수가 각각 0, 1, 2, 3, 4, 5 대가 나올 확률은?

In [16]:
N = 5
for i in range(N + 1):
    prob = stats.binom.pmf(k = i, n = N, p = 0.3)
    print("P(X={0}) = {1:.3f}".format(i, prob))

P(X=0) = 0.168
P(X=1) = 0.360
P(X=2) = 0.309
P(X=3) = 0.132
P(X=4) = 0.028
P(X=5) = 0.002


### 포아송 분포 : 한 상점에서는 평균 1시간에 3명 씩 포아송분포로 손님이 도착한다. 이 때 10시에서 11시 사이에 다음이 발생활 확률을 구하시오

* 2명의 손님이 도착
* 1명 이하의 손님이 도착
* 3명 이상의 손님이 도착

In [17]:
mu = 3
two_prob = stats.poisson.pmf(2, mu)
one_lower_prob = stats.poisson.cdf(1, mu)
three_upper_prob = 1 - stats.poisson.cdf(3, mu)

print("{0} 명의 손님이 도착할 확률 : {1:.2f}%".format(2, two_prob * 100))
print("{0} 명 이하의 손님이 도착할 확률 : {1:.2f}%".format(1, one_lower_prob * 100))
print("{0} 명 이상의 손님이 도착할 확률 : {1:.2f}%".format(1, three_upper_prob * 100))

2 명의 손님이 도착할 확률 : 22.40%
1 명 이하의 손님이 도착할 확률 : 19.91%
1 명 이상의 손님이 도착할 확률 : 35.28%


### 정규성 검정

In [31]:
ds_2sam = pd.read_csv("Datas/2sample_t_test.csv", engine = "python")
ds_2sam.head()

,Mat_A,Mat_B,Total,Sub
0,73.4,68.7,73.4,1
1,77.0,71.4,77.0,1
2,73.7,69.8,73.7,1
3,73.3,75.3,73.3,1
4,73.1,71.3,73.1,1


In [32]:
ds_2sam = ds_2sam.dropna()
data1 = ds_2sam['Mat_A']
data2 = ds_2sam['Mat_B']
print(shapiro(data1))
print(shapiro(data2))

ShapiroResult(statistic=0.9062636494636536, pvalue=0.2562994360923767)
ShapiroResult(statistic=0.9729753732681274, pvalue=0.9169777035713196)


### 새로운 품종의 감자 20 Box 를 랜덤추출했다.(모표준편차 = 3.8) 이 자료로부터 신품종 배추의 평균 무게에 대한 95% 신뢰 구간을 구하시오.

In [33]:
df = pd.DataFrame({"sample" : [18, 18, 20, 21, 20, 23, 19, 18, 17, 21, 22, 20, 20, 21, 20, 19, 19, 18, 17, 19]})
df.head()
lower, upper = stats.norm.interval(0.95, loc = np.mean(df), scale = 3.8 / np.sqrt(20))
print("신뢰 구간 :({0}, {1})".format(lower.round(2), upper.round(2)))

신뢰 구간 :([17.83], [21.17])


### 모표준편차 모르는 경우 : 제품 개발자는 새로 개발된 강종의 평균 인장강도를 70kg/mm으로 기대하고 있음. 표본평균은 71.50Kg/mm, 표본표준편차는 1.933Kg/mm 이 모평균값의 95%신뢰구간을 추정하시오

In [46]:
df = pd.DataFrame({"sample" : [73, 71, 74, 69, 70, 73, 70, 68, 75, 72, 70, 72, 73, 70, 70, 72, 71, 70, 75, 72]})
lower, upper = stats.t.interval(0.95, len(df) - 1, loc = np.mean(df), scale = scipy.stats.sem(df))
print("신뢰 구간 :({0}, {1})".format(lower.round(2), upper.round(2)))

신뢰 구간 :([70.6], [72.4])


### 모분산 신뢰구간 추정

In [41]:
n = 50
s = 4.68
kai_1 = 71.42
kai_2 = 32.36
lower = (n - 1) * s**2 / kai_1
upper = (n - 1) * s**2 / kai_2

print("모분산의 95% 신뢰 구간 :({0:.2f}, {1:.2f})".format(lower, upper))

모분산의 95% 신뢰 구간 :(15.03, 33.16)


### 모비율 신뢰구간 추정 : 통화관리 성공율

In [39]:
phat = 1 - 9 / 50
n = 50
Z = 1.96
lower = phat - Z * np.sqrt(phat * (1 - phat) / n)
upper = phat + Z * np.sqrt(phat * (1 - phat) / n)

print("통화 관리 성공율에 대한 95% 신뢰 구간 :({0}, {1})".format(lower.round(2), upper.round(2)))

통화 관리 성공율에 대한 95% 신뢰 구간 :(0.71, 0.93)


### 고객 만족도 데이터 분석을 통한 만족도의 변화

In [43]:
df = pd.DataFrame({'sample' : [85.0,79.0,79.1,79.9,81.6,78.6,85.4,83.4,78.1,79.2]})
t_result = stats.ttest_1samp(df, 78.0)

t, p = t_result.statistic.round(3), t_result.pvalue.round(3)

print("1-Sample t-test")
print("t 통계량 : {}".format(t))
print("p-value : {}".format(p))

1-Sample t-test
t 통계량 : [3.379]
p-value : [0.008]


### 제품을 A공급자와 B공급자로 부터 받아 두께를 측정해서 다음의 데이터를 얻었다. 공급자 간의 모평균에 차이가 있을 수 있는가?

In [49]:
df1 = pd.DataFrame({'sample': [1.883, 1.715, 1.799, 1.768, 1.711, 1.832, 1.427, 1.344]})
df2 = pd.DataFrame({'sample': [1.435, 1.572, 1.486, 1.511, 1.457, 1.548, 1.404, 1.883]})

result = stats.ttest_ind(df1, df2)
t, p = result.statistic.round(3), result.pvalue.round(3)

print("2-Sample test")
print("t:{}".format(t))
print("p:{}".format(p))

2-Sample test
t:[1.7]
p:[0.111]


### 회사원의 영어 점수 차이 점검하기

In [52]:
df1 = pd.DataFrame({'sample': [720, 589, 780, 648, 720, 589, 780, 648, 780, 648]})
df2 = pd.DataFrame({'sample': [710, 580, 787, 712, 750, 600, 782, 670, 790, 680]})
result = stats.ttest_rel(df1, df2)
t, p = result.statistic.round(3), result.pvalue.round(3)

print("Paired test")
print("t:{}".format(t))
print("p:{}".format(p))

Paired test
t:[-2.266]
p:[0.05]


### 동전이 정상적인가를 검증해보자

In [57]:
count1 = 40
n_obs1 = 100
value1 = 0.5
stat, pval = proportions_ztest(count, n_obs, value)

print("Proportion test")
print("z:{0:0.3f}".format(stat))
print("p:{0:0.3f}".format(pval))

Proportion test
z:-2.041
p:0.041


### 두 공장에서의 불량률을 측정해보자

In [59]:
counts = np.array([14,5])
nobs = np.array([1200, 1200])
stat, pval = proportions_ztest(counts, nobs)

print("Proportion test")
print("z:{0:0.3f}".format(stat))
print("p:{0:0.3f}".format(pval))

Proportion test
z:2.073
p:0.038
